In [1]:
import torch
import numpy as np
from datasets import load_from_disk
from transformers import AutoModel, AutoConfig
import sentencepiece as spm
from sklearn.metrics.pairwise import cosine_similarity


C:\Users\tanuj\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from datasets import load_from_disk

DATA_PATH = "bhagavad_gita_qa_sanskrit_partial"   # 🔥 FOLDER, not file

dataset = load_from_disk(DATA_PATH)

print("Dataset loaded:", dataset)
print("Number of rows:", len(dataset))
print("Sample row:\n", dataset[0])


Dataset loaded: Dataset({
    features: ['chapter_no', 'verse_no', 'question', 'answer', 'question_sa', 'answer_sa'],
    num_rows: 842
})
Number of rows: 842
Sample row:
 {'chapter_no': 1, 'verse_no': 1, 'question': 'Why does Dhritarashtra ask Sanjaya to describe the battlefield?', 'answer': 'Dhritarashtra is blind, both physically and symbolically — representing ignorance. He wants Sanjaya to narrate what is happening because he is anxious about the outcome of the war and whether his sons (Kauravas) will win.', 'question_sa': 'धृतराष्ट्रः सञ्जयं युद्धक्षेत्रस्य वर्णनं किमर्थं प्रार्थयति?', 'answer_sa': 'धृतराष्ट्रः अन्धः अस्ति, शारीरिकरूपेण प्रतीकात्मकरूपेण च-अज्ञानस्य प्रतिनिधित्वं करोति। सः इच्छति यत् सञ्जयः किं घटयति इति वर्णयतु, यतः सः युद्धस्य परिणामस्य विषये चिन्तितः अस्ति, तस्य पुत्राः (कौरवाः) विजयन्ति वा इति च।'}


In [4]:
sp = spm.SentencePieceProcessor()
sp.load("sp_unigram_64k.model")

print("Tokenizer loaded.")


Tokenizer loaded.


In [5]:
MODEL_PATH = "bert-sanskrit-light"

print("Loading SanskritBERT model...")

config = AutoConfig.from_pretrained(MODEL_PATH)
model = AutoModel.from_pretrained(MODEL_PATH)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(DEVICE)
model.eval()

print("Model loaded on", DEVICE)


Loading SanskritBERT model...


Some weights of BertModel were not initialized from the model checkpoint at bert-sanskrit-light and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded on cpu


In [6]:
def encode(text):
    # Tokenize with your sentencepiece
    pieces = sp.encode_as_pieces(text)
    ids = sp.encode_as_ids(text)

    input_ids = torch.tensor([ids], dtype=torch.long).to(DEVICE)
    attention_mask = torch.ones_like(input_ids).to(DEVICE)

    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

    # Use mean pooling over token embeddings
    token_embeddings = outputs.last_hidden_state  # (1, seq_len, hidden)
    sentence_embedding = token_embeddings.mean(dim=1)  # (1, hidden)

    return sentence_embedding.cpu().numpy()[0]


In [7]:
print("\nEncoding all Sanskrit questions...")

question_texts = dataset["question_sa"]
answer_texts   = dataset["answer_sa"]

question_embeddings = []

for i, q in enumerate(question_texts):
    emb = encode(q)
    question_embeddings.append(emb)

    if i % 50 == 0:
        print(f"Encoded {i}/{len(question_texts)} questions...")

question_embeddings = np.array(question_embeddings)

print("\nAll question embeddings ready.")
print("Embedding shape:", question_embeddings.shape)



Encoding all Sanskrit questions...
Encoded 0/842 questions...
Encoded 50/842 questions...
Encoded 100/842 questions...
Encoded 150/842 questions...
Encoded 200/842 questions...
Encoded 250/842 questions...
Encoded 300/842 questions...
Encoded 350/842 questions...
Encoded 400/842 questions...
Encoded 450/842 questions...
Encoded 500/842 questions...
Encoded 550/842 questions...
Encoded 600/842 questions...
Encoded 650/842 questions...
Encoded 700/842 questions...
Encoded 750/842 questions...
Encoded 800/842 questions...

All question embeddings ready.
Embedding shape: (842, 256)


In [8]:
def query_model(query, top_k=5):
    print("\n==============================")
    print("QUERY:", query)
    print("==============================")

    query_emb = encode(query)

    # Cosine similarity with all stored questions
    sims = cosine_similarity([query_emb], question_embeddings)[0]

    # Top K matches
    top_indices = np.argsort(sims)[::-1][:top_k]

    print("\n🔝 Top Matches:\n")

    for rank, idx in enumerate(top_indices, 1):
        print(f"--- Rank {rank} | Similarity: {sims[idx]:.4f} ---")
        print("Matched Question:", question_texts[idx])
        print("Answer:", answer_texts[idx][:300], "...")
        print()


In [9]:
query = "धृतराष्ट्रः संजयम् युद्धभूमेः वर्णनं किमर्थं पृच्छति?"

query_model(query, top_k=5)



QUERY: धृतराष्ट्रः संजयम् युद्धभूमेः वर्णनं किमर्थं पृच्छति?

🔝 Top Matches:

--- Rank 1 | Similarity: 0.8822 ---
Matched Question: धृतराष्ट्रः सञ्जयं युद्धक्षेत्रस्य वर्णनं किमर्थं प्रार्थयति?
Answer: धृतराष्ट्रः अन्धः अस्ति, शारीरिकरूपेण प्रतीकात्मकरूपेण च-अज्ञानस्य प्रतिनिधित्वं करोति। सः इच्छति यत् सञ्जयः किं घटयति इति वर्णयतु, यतः सः युद्धस्य परिणामस्य विषये चिन्तितः अस्ति, तस्य पुत्राः (कौरवाः) विजयन्ति वा इति च। ...

--- Rank 2 | Similarity: 0.8170 ---
Matched Question: पद्ये पाण्डवानां सेनायाः वर्णनं कथं कृतम्?
Answer: पाण्डवानां सेना युद्धाय सज्जः इति वर्णितम् अस्ति, यत् ते सामरिकरूपेण व्यवस्थिताः, युद्धाय सज्जाः च इति सूचयति। ...

--- Rank 3 | Similarity: 0.8075 ---
Matched Question: 'युद्धे सर्वदा विजयी'इति कृपेः वर्णनं कियत् महत्त्वम् अस्ति?
Answer: 'युद्धे सर्वदा विजयी'इति वर्णितं कृपाः, तस्य कौशलं, युद्धे सफलतां च प्रकाशयति। एतत् दुर्भेद्यः योद्धा इति तस्य प्रतिष्ठां उद्घाटयति तथा च सः कौरवसेनायाः प्रमुखः सम्पत्तिः अस्ति इति सूचयति। एतत् विवरणं तेषां शक्तिं, क्षमतां च आत

In [10]:
query = "किमर्थं संजयम् धृतराष्ट्रः युद्धभूमेः वर्णनं पृच्छति?"
query_model(query, top_k=3)


QUERY: किमर्थं संजयम् धृतराष्ट्रः युद्धभूमेः वर्णनं पृच्छति?

🔝 Top Matches:

--- Rank 1 | Similarity: 0.8453 ---
Matched Question: धृतराष्ट्रः सञ्जयं युद्धक्षेत्रस्य वर्णनं किमर्थं प्रार्थयति?
Answer: धृतराष्ट्रः अन्धः अस्ति, शारीरिकरूपेण प्रतीकात्मकरूपेण च-अज्ञानस्य प्रतिनिधित्वं करोति। सः इच्छति यत् सञ्जयः किं घटयति इति वर्णयतु, यतः सः युद्धस्य परिणामस्य विषये चिन्तितः अस्ति, तस्य पुत्राः (कौरवाः) विजयन्ति वा इति च। ...

--- Rank 2 | Similarity: 0.7971 ---
Matched Question: पद्ये पाण्डवानां सेनायाः वर्णनं कथं कृतम्?
Answer: पाण्डवानां सेना युद्धाय सज्जः इति वर्णितम् अस्ति, यत् ते सामरिकरूपेण व्यवस्थिताः, युद्धाय सज्जाः च इति सूचयति। ...

--- Rank 3 | Similarity: 0.7925 ---
Matched Question: 'युद्धे सर्वदा विजयी'इति कृपेः वर्णनं कियत् महत्त्वम् अस्ति?
Answer: 'युद्धे सर्वदा विजयी'इति वर्णितं कृपाः, तस्य कौशलं, युद्धे सफलतां च प्रकाशयति। एतत् दुर्भेद्यः योद्धा इति तस्य प्रतिष्ठां उद्घाटयति तथा च सः कौरवसेनायाः प्रमुखः सम्पत्तिः अस्ति इति सूचयति। एतत् विवरणं तेषां शक्तिं, क्षमतां च आत

In [11]:
query = "कर्मयोगस्य महत्त्वं किम्?"
query_model(query, top_k=5)



QUERY: कर्मयोगस्य महत्त्वं किम्?

🔝 Top Matches:

--- Rank 1 | Similarity: 0.8701 ---
Matched Question: पद्ये उल्लिखितस्य रथस्य महत्त्वं किम्?
Answer: अयं रथः वैभवयुतः, श्वेत-अश्वैः आहृतः च इति वर्णितः अस्ति, यः पवित्रतायाः, दैविक-अनुग्रहस्य च प्रतीकः अस्ति। एतत् श्रीकृष्णस्य अर्जुनस्य च भव्यतां, दैविक-समर्थनं च प्रकाशयति। ...

--- Rank 2 | Similarity: 0.8322 ---
Matched Question: अस्मिन् श्लोकस्य वृष्ट्या यज्ञस्य (यज्ञस्य) संयोगस्य महत्त्वं किम्?
Answer: यज्ञं वृष्ट्या सह सम्बद्ध्वा, श्लोकः निःस्वार्थ-क्रियायाः, अनुष्ठानस्य च महत्त्वं जगते सामञ्जस्यस्यस्य साधनरूपेण प्रकाशयति। यज्ञाः ईश्वरीयशक्तिं प्रसन्नं कुर्वन्तः नैवेद्यरूपेण दृश्यन्ते, ये क्रमेण भूमिं वृष्ट्या आशीर्ब्बादयन्ति, जीवनस्य अनुवर्तनं सुनिश्चितं कुर्वन्ति। ...

--- Rank 3 | Similarity: 0.8222 ---
Matched Question: अस्मिन् पद्ये शरीरस्य वस्त्रस्य तुलनायाः महत्त्वं किम् अस्ति?
Answer: शरीरस्य वस्त्रैः सह तुलनां भौतिकरूपस्य अस्थायीं बाह्यं च स्वरूपं सूचयति, यथा परिवर्तनीयं वस्त्रम्। एतत् सादृश्यं आत्मः शाश्वतः अस्ति, शरीरस्